In [ ]:
import numpy as np
from bokeh.plotting import figure
from bokeh.models import LabelSet
import datetime as dt
import panel as pn
import concurrent.futures
from bokeh.models import ColumnDataSource
pn.extension()
import panel as pn
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import json 
import pandas as pd 
# from pandas.json_normalize import json_normalize #package for flattening json in pandas df

import pathlib

import holoviews as hv
import matplotlib.pyplot as plt
import hvplot
import param
import hvplot.pandas
from bokeh.models import GlyphRenderer, LinearAxis, LinearScale, Range1d
from datetime import date, time

pn.extension(sizing_mode='stretch_width')

pn.extension('perspective')
from cognite.client.utils import MIN_TIMESTAMP_MS, MAX_TIMESTAMP_MS

### Configure cognite client

In [2]:
# Contact Project Administrator to get these
TENANT_ID = "3aa4a235-b6e2-48d5-9195-7fcf05b459b0" # AAD using @equinor.com
CLIENT_ID = "3d7a85da-05ab-4675-b5f9-0c4b96a1c84b" # CDF_ALL_USER_APPLICATION_FLOW
CDF_CLUSTER = "westeurope-1"
# -------------for DEV project-----------------
COGNITE_PROJECT = "equinor-dev"

# -------------for TEST project-----------------
# COGNITE_PROJECT = "equinor-test"


from cognite.client import CogniteClient
client = CogniteClient.default_oauth_interactive(
    project=COGNITE_PROJECT,
    tenant_id=TENANT_ID,
    client_id=CLIENT_ID,
    cdf_cluster=CDF_CLUSTER,
    client_name="my-cognite-python-test", # a name to identify your session
)
from cognite.client.data_classes import LabelFilter



### Define Dataset

In [3]:
#  
wellcom_dataset = "src:015:wellcom"
witsml_dataset = "src:014:witsmldata"
active_wellbore = "active:wellbore"

#list wellcom wellbores that are active in WITSML
active_wellbores = client.assets.list(data_set_external_ids = wellcom_dataset, labels=LabelFilter(contains_all=[active_wellbore]), limit=-1)
#Get sensor data 
witsml_db  = client.time_series.list(data_set_external_ids = witsml_dataset, limit=-1)

### Convert to pandas dataframe and flatten

In [4]:
# Wellbores
pd_active_wellbores = active_wellbores.to_pandas(metadata_prefix=active_wellbores)

# Sensordata
_pd_witsml = witsml_db.to_pandas(metadata_prefix=witsml_db)
_pd_witsml.rename(columns ={'name': 'sensorName'} , inplace = True)
# Flatten Sensordata
_pd_witsml_flattened = pd.concat([_pd_witsml.drop(['metadata'], axis=1), pd.json_normalize(_pd_witsml['metadata'] )], axis=1)

#Limiting columns
_pd_witsml_flattened_lim = _pd_witsml_flattened[['id', 'uidWell', 'sensorName', 'name', 'uidWellbore',  'nameWellbore','external_id',  'is_string', 'asset_id', 'is_step',
       'description', 'security_categories', 'data_set_id', 'created_time',
       'last_updated_time', 'type', 'commonData_dTimCreation',
       'commonData_dTimLastChange', 'commonData_itemState',
       'commonData_sourceName', 'creationDate', 'direction',
       'endDateTimeIndex', 'indexCurve', 'indexType',  'nameWell',
       'nullValue', 'objectGrowing', 'serviceCompany',
       'startDateTimeIndex',  'uidLog',
       'unavailable_in_source', 'witsml-server-ref', 'source',
       'curveDescription', 'maxDateTimeIndex', 'minDateTimeIndex', 'mnemAlias',
       'mnemonic', 'typeLogData', 'uid']]





In [5]:

#Get name and id of wellbore
pd_wellbores = _pd_witsml_flattened[['nameWellbore', 'uidWellbore']]
#Get rid of duplicates
pd_wellbores = pd_wellbores.drop_duplicates(subset=['uidWellbore'])
#Create dictionary 
wellbore_map = {row['nameWellbore']: row['uidWellbore'] for index, row in pd_wellbores.iterrows()}

#Get name and id of sensor
pd_sensors = _pd_witsml_flattened[['sensorName', 'id', 'external_id']]
#Get rid of duplicates
pd_sensors = pd_sensors.drop_duplicates(subset=['id'])
 

# Main Dashboard

In [ ]:

pn.param.ParamMethod.loading_indicator = True

# Defining the paramterised class 
class CustomDashboard(param.Parameterized):
    
    #Parameter for wellbore selector 
    wellbore = param.ObjectSelector(default=pd_wellbores['nameWellbore'].iloc[0], objects=list( sorted(pd_wellbores['nameWellbore'])    ))
    #Dropdown for senors
    sensor = param.ObjectSelector(default= _pd_witsml_flattened[_pd_witsml_flattened['nameWellbore'] == pd_wellbores['nameWellbore'].iloc[0] ]['sensorName'].iloc[0]   , objects=list( _pd_witsml_flattened[_pd_witsml_flattened['nameWellbore'] == pd_wellbores['nameWellbore'].iloc[0] ]['sensorName']) ) 
    #This is a checkbox that allows the user to view all sensor data, off by default because it's very slow
    show_all_dates = param.Boolean(default = False)
    #Date picker
    date_of_operations = param.CalendarDate(default = date.today())
        

    # @param.depends('show_all_dates', watch=True)
    # def toggle_on_change(self) :
    #     self.plot()
    

    @param.depends('wellbore', watch=True)
    def update_sensors(self):
        wellbore_id = wellbore_map[self.wellbore]
        filtered_sensors = _pd_witsml_flattened[_pd_witsml_flattened['uidWellbore'] == wellbore_id]
        sensor_names = list(sorted(filtered_sensors['sensorName']))
        self.param['sensor'].objects = sensor_names
        if sensor_names:
                self.sensor = sensor_names[0]
        else:
            self.sensor= None
    
    
    def get_sensor_id(self):
        wellbore_id= wellbore_map[self.wellbore]
        sensor_data = _pd_witsml_flattened[(_pd_witsml_flattened['uidWellbore'] == wellbore_id) & (_pd_witsml_flattened['sensorName'] == self.sensor)]
        if not sensor_data.empty:
            return sensor_data['id'].iloc[0]
        return None
    

    @param.depends('sensor') 
    def view(self):
        sensor_id = self.get_sensor_id()
        if sensor_id:
                return pn.pane.Str(f'Selected Sensor ID: {sensor_id}')
        return pn.pane.Str('No sensor selected')
    


    #Main function for plotting. Too long, should be refactored with more functions
    @param.depends('sensor', 'date_of_operations' , 'show_all_dates')
    def plot(self):

        if self.sensor:

            if(self.show_all_dates):
                print("Show all dates is now checked and timestamps are set to cognite utils.") 
                print(self.show_all_dates)
                min_timestamps_ms =MIN_TIMESTAMP_MS
                max_timestamps_ms = MAX_TIMESTAMP_MS
            #If user is not viewing all the dates, then min and max should be min and max of 24 hours from the selected date.    
            else:
                print("Show all dates is set to false and timestamps are set to selected date") 
                #Getting time stamps max and min of 24hrs of selected date
                #Keeping datetime as well as timestamps, datetime is used later when filtering out operations data
                min_datetime = datetime.combine(self.date_of_operations, time.min)
                min_timestamp =min_datetime.timestamp()
                max_datetime = datetime.combine((self.date_of_operations + timedelta(hours = 24)) , time.min)
                max_timestamp = max_datetime.timestamp()
                #Need to be milliseconds to work with cognite
                min_timestamps_ms = int(min_timestamp *1000 )
                max_timestamps_ms = int(  max_timestamp * 1000)
                
            
            #Getting sensor id
            sensor_id = int(self.get_sensor_id())
            sensor_data = client.time_series.data.retrieve(id=sensor_id,start=min_timestamps_ms,  end=max_timestamps_ms + 1)
            
            #Converting sensor data to pandas
            sensor_data_to_pd = sensor_data.to_pandas()
            # Reindexing the sensor data
            sensor_data_to_pd['Value'] = sensor_data_to_pd[sensor_data_to_pd.columns]
            sensor_data_to_pd['Date'] = pd.to_datetime(sensor_data_to_pd.index)
            #Adding a more consistant series of time stamps
            
            #Skips this part if  user is not viewing all date
            if (self.show_all_dates):
                start_date = sensor_data_to_pd['Date'].min()
                end_date = sensor_data_to_pd['Date'].max()
                #Finding the smallest interval to define the frequency
            else:
                 start_date = self.date_of_operations
                 end_date = self.date_of_operations + timedelta(hours = 24)
                 print("This should run. Have a look at start date and end date:")
                 print(start_date)
                 print(end_date)
            
            time_diff = sensor_data_to_pd['Date'].diff().min()         



            if(pd.isnull (time_diff)):
                 print("There is no time diff, setting frequency to seconds")
                 time_diff = 'S'
            else:
                print("debug else")
                print(time_diff)     

            #Generate timestamps
            new_timestamps = pd.date_range(start=start_date, end=end_date, freq= time_diff)

            
            print("Let's check the smallest generated timestamp: " )
            print(new_timestamps.min)
            
            #Create a new DataFrame with the new timestamps
            new_data = pd.DataFrame(new_timestamps, columns=['New_Timestamps'])
            #Merge the new data with the original data using an outer join
            sensor_data_to_pd = pd.merge(sensor_data_to_pd, new_data, how='outer', left_on='Date', right_on='New_Timestamps')
            #Joining the new timestamps 
            sensor_data_to_pd.set_index('New_Timestamps')             

            if not sensor_data_to_pd.empty:
                
                
                max_sensor_value = sensor_data_to_pd['Value'].max()
                min_sensor_value = sensor_data_to_pd['Value'].min()
                center =sensor_data_to_pd['Value'].median()
                sensor_plot = sensor_data_to_pd.hvplot.line(x='Value', y='New_Timestamps', autorange="y", title=f'Sensor Data for {self.sensor}').opts(invert_yaxis=True, width = 800, height = 1000)
                uptime_downtime_data = [
                    {'Start': '2022-11-25 05:00:00', 'End': '2022-11-25 05:30:00', 'Status': 'Uptime'},
                    {'Start': '2022-11-25 06:12:30', 'End': '2022-11-25 06:45:30', 'Status': 'Downtime'},
                    {'Start': '2022-11-25 08:12:30', 'End': '2022-11-25 10:12:30', 'Status': 'Uptime'},

                    {'Start': '2022-11-28 05:00:00', 'End': '2022-11-28 05:30:00', 'Status': 'Downtime'},
                    {'Start': '2022-11-28 06:12:30', 'End': '2022-11-28 06:45:30', 'Status': 'Uptime'},
                    {'Start': '2022-11-28 08:12:30', 'End': '2022-11-28 10:12:30', 'Status': 'Uptime'},
                    # Add more uptime/downtime periods as needed
                ]

                

                uptime_downtime_df = pd.DataFrame(uptime_downtime_data)

                uptime_downtime_df['Start'] = pd.to_datetime(uptime_downtime_df['Start'])
                uptime_downtime_df['End'] = pd.to_datetime(uptime_downtime_df['End'])


                  
                if(not(self.show_all_dates)):
                    #Filtering out the uptime_downtime data to remove data that is no within the range of the selected date
                    uptime_downtime_df = uptime_downtime_df[(uptime_downtime_df['Start'] >= min_datetime) & (uptime_downtime_df['End'] <= max_datetime)]
                
                labels = []
                rects = []
                for _, row in uptime_downtime_df.iterrows():
                    midpoint = row['Start'] + (row['End'] - row['Start']) /2
                    labels.append(hv.Text( center, midpoint, row['Status'] ).opts(text_font_style = 'bold', text_align = 'left') )
                    color = 'green' if row['Status'] == 'Uptime' else 'red'
                    rects.append(hv.Rectangles((min_sensor_value,row['Start'],max_sensor_value , row['End'])).opts(color=color, alpha=0.2))
                
                rects_overlay = hv.Overlay(rects)
                labels_overlay = hv.Overlay(labels)
        
                return  sensor_plot *rects_overlay * labels_overlay
        
        
        return pn.pane.Str('No data available for the selected sensor')

    
    #Run update sensor method to populate sensor dropdown
    def __init__(self, **params):
        super().__init__(**params)
        self.update_sensors()
        


    def panel(self):
        return pn.Row(
            pn.Column(pn.Param(self.param) , self.view  , width = 300), pn.Row(self.plot)

        )

# Instantiate the dashboard
dashboard = CustomDashboard()

# Trigger initial sensor update
#dashboard.update_sensors()

# Display the dashboard
dashboard.panel().show()



